In [116]:
import pandas as pd
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import Counter

# 데이터를 읽어봅시다.
train_data = pd.read_table('~/aiffel/sentiment_classification/data/ratings_train.txt')
test_data = pd.read_table('~/aiffel/sentiment_classification/data/ratings_test.txt')

### 데이터 로더 구성

    데이터의 중복 제거
    NaN 결측치 제거
    한국어 토크나이저로 토큰화
    불용어(Stopwords) 제거
    사전word_to_index 구성
    텍스트 스트링을 사전 인덱스 스트링으로 변환
    X_train, y_train, X_test, y_test, word_to_index 리턴

차례대로 해보자

1. 데이터의 중복 제거

In [117]:
from konlpy.tag import Mecab
tokenizer = Mecab()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [118]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [119]:
train_data['document'].value_counts()

굿                             181
good                           92
최고                             85
쓰레기                            79
별로                             66
                             ... 
순수하게 클릭했다가 혀만 차고 껐다             1
쓰레기같은영화..                       1
이딴 쓰레기 뭐좋다고 계속 만드냐..집어쳐라..      1
이런 OOO                          1
역시 일본식 발상, 호기심보단 역겨움이 밀려온다      1
Name: document, Length: 146182, dtype: int64

굿, good 과 같은 평가를 남긴 document들이 많다. 중복되는 것은 지워주고, 지운 인덱스는 초기화해준다.

In [120]:
train_data.drop_duplicates(subset=['document'], inplace=True, ignore_index=True)

In [121]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146183 entries, 0 to 146182
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        146183 non-null  int64 
 1   document  146182 non-null  object
 2   label     146183 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.3+ MB


중복된 행을 제외하고 146183개의 행이 남았다. null값이 있는 한 행마저 제거해보자

In [122]:
train_data.dropna(axis=0, inplace=True)

In [123]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146182 entries, 0 to 146182
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        146182 non-null  int64 
 1   document  146182 non-null  object
 2   label     146182 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.5+ MB


In [124]:
train_data.tail()

,id,document,label
146178,6222902,인간이 문제지.. 소는 뭔죄인가..,0
146179,8549745,평점이 너무 낮아서...,1
146180,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
146181,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1
146182,9619869,한국 영화 최초로 수간하는 내용이 담긴 영화,0


In [125]:
train_data.isnull().sum()

id          0
document    0
label       0
dtype: int64

시간 남으면 해볼 것.

행을 drop_duplicates에서 first나 last 기준으로 제거하는 것이 아니라 label이 더 많은것을 제

2. Stopwords 제거

In [128]:
import re

In [139]:
for i, x in enumerate(train_data['document'].head(5)):
    print(x)
    _str = re.sub(r"[^a-zA-Z0-9]","",x)
    print(_str)

아 더빙.. 진짜 짜증나네요 목소리

흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나

너무재밓었다그래서보는것을추천한다

교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정

사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다



...같은